# Setup

Remember to install CUDA dependencies

In [1]:
import glob
import os
import shutil
import re
import random
seed = 420
random.seed(seed)

from ultralytics import YOLO, settings
root_dir = os.getcwd().replace('\\notebooks', '')
settings.update({'datasets_dir': f'{root_dir}/data/processed', 'runs_dir': f'{root_dir}/yolo/runs'})

from roboflow import Roboflow

# Sorting Kaggle and MAN datasets into train/val/test

Sources:
- https://www.kaggle.com/datasets/hamidl/yoloqrlabeled?resource=download
- https://app.roboflow.com/rs-xldmw/data-matrix-codes/

In [2]:
# create required directories
if not os.path.exists('../data/processed/'):
    os.mkdir('../data/processed')
    os.mkdir('../data/processed/images')
    os.mkdir('../data/processed/images/train')
    os.mkdir('../data/processed/images/val')
    os.mkdir('../data/processed/images/test')
    os.mkdir('../data/processed/labels')
    os.mkdir('../data/processed/labels/train')
    os.mkdir('../data/processed/labels/val')
    os.mkdir('../data/processed/images/test')

    # copy images and labels to required directories by train/val splits

    # total images
    total = len(glob.glob('../data/YOLO-QR-datasets/Dataset 1/images/*.jpg')) + \
            len(glob.glob('../data/YOLO-QR-datasets/Dataset 2/images/*.jpg')) + \
            len(glob.glob('../data/YOLO-QR-datasets/Dataset 3/images/*.jpg'))

    train = int(total * 0.9) # floored (we may miss our on 1 or 2 images but that's fine)
    val = int(total * 0.1)
    print(f'Train : {train}')
    print(f'Val   : {val}')

    # getting filepaths of all images and annotations
    filepaths = {'images': [], 'labels': []}
    for i in range(1, 4):
        # images
        for file in glob.glob(f'../data/YOLO-QR-datasets/Dataset {i}/images/*.jpg'):
            filepaths['images'].append(file)
        # labels
        for file in glob.glob(f'../data/YOLO-QR-datasets/Dataset {i}/labels/*.txt'):
            filepaths['labels'].append(file)

    print(filepaths['images'][:2])
    print(filepaths['labels'][:2])

    # shuffled indexes
    indexes = list(range(total))
    random.Random(seed).shuffle(indexes)
    print(indexes[:2])

    # copy images and labels to required directories with train/val splits
    pattern = r'\d+' # regex pattern for grabbing first number in label file
    for i, index in enumerate(indexes):
        if i < train:
            # train

            # copy image file as-is
            shutil.copyfile(filepaths['images'][index], f'../data/processed/images/train/{i}.jpg')

            # copy label file but change class index to 0
            with open(filepaths['labels'][index], 'r') as f:
                line = f.readline()
            num = re.search(pattern, line).group()
            line = line.replace(num, '0', 1)
            with open(f'../data/processed/labels/train/{i}.txt', 'w') as f:
                f.write(line)
        
        else:
            # val
            shutil.copyfile(filepaths['images'][index], f'../data/processed/images/val/{i}.jpg')
            with open(filepaths['labels'][index], 'r') as f:
                line = f.readline()
            num = re.search(pattern, line).group()
            line = line.replace(num, '0', 1)
            with open(f'../data/processed/labels/val/{i}.txt', 'w') as f:
                f.write(line)

    # download MAN dataset to tmp folder for later sorting
    if not os.path.exists('../data/processed/tmp'):
        os.mkdir('../data/processed/tmp')
    
    with open('../roboflow_api_key.txt') as f:
        api_key = f.read().strip()

    rf = Roboflow(api_key=api_key)
    project = rf.workspace('rs-xldmw').project('data-matrix-codes')
    version = project.version(2)
    dataset = version.download('yolov11', '../data/processed/tmp/')

    # move files to test folder
    for file in glob.glob('../data/processed/tmp/test/images/*.jpg'):
        shutil.move(file, '../data/processed/images/test/')
    for file in glob.glob('../data/processed/tmp/test/labels/*.txt'):
        shutil.move(file, '../data/processed/labels/test/')

    # remove tmp folder
    shutil.rmtree('../data/processed/tmp')

    # create yaml file for training YOLO on
    with open('../data/processed/dataset.yaml', 'w') as f:
        f.write(f'path: {root_dir}/data/processed/\n')
        f.write('train: images/train\n')
        f.write('val: images/val\n')
        f.write('test: images/test\n')
        f.write('\n')
        f.write('nc: 1\n')
        f.write('names: [\'Data Matrix\']\n')

# Training

## From Scratch

Took ~4.6hrs on rtx 2070

In [3]:
model = YOLO('yolo11n.yaml', task='detect') # build a new model from YAML

In [4]:
model.train(
    data='../data/processed/dataset.yaml', # path to yaml file which specifies dataset parameters
    epochs=100,
    imgsz=640,
    single_cls=True,                      # single class training
    patience=10,                          # early stopping patience (after this many epochs with no improvement stop training)
    pretrained=False,                     # don't use pre-trained weights
    plots=True,                           # create plots

    # solving GPU memory issue?
    workers=0,                            # number of worker threads for data loading (0 reduces memory problems at cost of slower training)
    batch = 8,                            # batch size (default 16, reducing to 8 can help)
)

New https://pypi.org/project/ultralytics/8.3.8 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.6  Python-3.12.5 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.yaml, data=../data/kaggle/kaggle-QR.yaml, epochs=100, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=0, project=None, name=train2, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save

100%|██████████| 5.35M/5.35M [00:00<00:00, 16.3MB/s]


AMP: checks passed 


train: Scanning C:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\data\kaggle\labels\train... 18310 images, 0 backgrounds, 0 corrupt: 100%|██████████| 18310/18310 [00:11<00:00, 1541.48it/s]


train: New cache created: C:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\data\kaggle\labels\train.cache


...

                   all       2035       2035      0.996      0.996      0.995      0.994
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 24, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



34 epochs completed in 4.615 hours.
Optimizer stripped from c:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\yolo\runs\detect\train2\weights\last.pt, 5.5MB
Optimizer stripped from c:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\yolo\runs\detect\train2\weights\best.pt, 5.5MB

Validating c:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\yolo\runs\detect\train2\weights\best.pt...
WARNING  validating an untrained model YAML will result in 0 mAP.
Ultralytics 8.3.6  Python-3.12.5 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:23<00:00,  5.52it/s]


                   all       2035       2035      0.996      0.996      0.995      0.995
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to c:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\yolo\runs\detect\train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000011AD7341490>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [7]:
# move best model to models directory
shutil.move('../yolo/runs/detect/train/weights/best.pt', '../yolo/models/kaggle_scratch.pt')
del model # flush memory

## Ultralytics Pretrained

In [8]:
model = YOLO('../yolo/models/yolo11n.pt', task='detect') # using ultralytics pre-trained model (on COCO dataset)

In [9]:
model.train(
    data='../data/processed/dataset.yaml', # path to yaml file which specifies dataset parameters
    epochs=100,
    imgsz=640,
    single_cls=True,                      # single class training
    patience=10,                          # early stopping patience (after this many epochs with no improvement stop training)
    pretrained=False,                     # don't use pre-trained weights
    plots=True,                           # create plots

    # solving GPU memory issue?
    workers=0,                            # number of worker threads for data loading (0 reduces memory problems at cost of slower training)
    batch = 8,                            # batch size (default 16, reducing to 8 can help)
)

New https://pypi.org/project/ultralytics/8.3.9 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.6  Python-3.12.5 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)
engine\trainer: task=detect, mode=train, model=../yolo/models/yolo11n.pt, data=../data/processed/dataset.yaml, epochs=100, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=0, project=None, name=train2, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, sh

RuntimeError: Dataset '../data/processed/dataset.yaml' error  '../data/processed/dataset.yaml' does not exist